# Simple DCGAN on the mnist dataset
This code was made after reading both these tutorials, alot of the code is inspired and even copied from them:
- https://towardsdatascience.com/gan-by-example-using-keras-on-tensorflow-backend-1a6d515a60d0
- https://www.kdnuggets.com/2016/07/mnist-generative-adversarial-model-keras.html

## Loading the depndencies

In [3]:
%matplotlib inline
# %matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt

import keras
from keras.layers import Dense, Activation, Dropout, Flatten, Reshape, LeakyReLU 
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D, BatchNormalization
from keras.datasets import mnist
from keras.models import Sequential
from keras.optimizers import RMSprop

from sklearn.utils import shuffle
%autosave 120

Using TensorFlow backend.


Autosaving every 120 seconds


## Loading and preparing the dataset

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data() #loading the mnist datatset

The image data needs to be normalized, a simple way of doing that in dividing by 255:

In [5]:
print(x_train.max()) #checking the max value in the training data
x_train = x_train.astype("float32")/255.  #normalizing the entire training data
print(x_train.max())    #then the checking the max after normalization

255
1.0


Althought the images are greyscale, we will add a channel dimension of 1 because the convolution layers require them: 

In [6]:
#adding the channel dimension
x_train = x_train.reshape((x_train.shape[0],x_train.shape[1],x_train.shape[2],1))
x_test = x_test.reshape((x_test.shape[0],x_test.shape[1],x_test.shape[2],1))

y_train = y_train.reshape((y_train.shape[0],-1))
y_test = y_test.reshape((y_test.shape[0],-1))

#saving the data shapes
train_data_shape = x_train.shape
test_data_shape = x_test.shape

train_label_shape = y_train.shape
test_label_shape = y_test.shape

#printing the shapes
print("train data shape:",train_data_shape)
print("train label shape:",train_label_shape)
print("test data shape:",test_data_shape)
print("test label shape:",test_label_shape)

train data shape: (60000, 28, 28, 1)
train label shape: (60000, 1)
test data shape: (10000, 28, 28, 1)
test label shape: (10000, 1)


## Creating the neural network models
First the Discriminator model is made, its a CNN the has one output that calssifies either tha data is real (1) or is fake (0):

In [7]:
D = Sequential()

depth = 64

dropout = 0.4

# In: 28 x 28 x 1, depth = 1

# Out: 14 x 14 x 1, depth=64

input_shape = train_data_shape[1:]

D.add(Conv2D(depth*1, 5, strides=2, input_shape=input_shape,padding='same'))
D.add(LeakyReLU(alpha=0.2))

D.add(Dropout(dropout))

D.add(Conv2D(depth*2, 5, strides=2, padding='same'))
D.add(LeakyReLU(alpha=0.2))

D.add(Dropout(dropout))

D.add(Conv2D(depth*4, 5, strides=2, padding='same'))
D.add(LeakyReLU(alpha=0.2))

D.add(Dropout(dropout))

D.add(Conv2D(depth*8, 5, strides=1, padding='same'))
D.add(LeakyReLU(alpha=0.2))

D.add(Dropout(dropout))

# Out: 1-dim probability

D.add(Flatten())

D.add(Dense(1))

D.add(Activation('sigmoid'))

D.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 64)        1664      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 128)         204928    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 7, 7, 128)         0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 256)         819456    
__________

In [8]:
G = Sequential()

dropout = 0.4

depth = 64+64+64+64

dim = 7

# In: 100

# Out: dim x dim x depth

G.add(Dense(dim*dim*depth, input_dim=100))

G.add(BatchNormalization(momentum=0.9))

G.add(Activation('relu'))

G.add(Reshape((dim, dim, depth)))

G.add(Dropout(dropout))

# In: dim x dim x depth

# Out: 2*dim x 2*dim x depth/2

G.add(UpSampling2D())

G.add(Conv2DTranspose(int(depth/2), 5, padding='same'))

G.add(BatchNormalization(momentum=0.9))

G.add(Activation('relu'))

G.add(UpSampling2D())

G.add(Conv2DTranspose(int(depth/4), 5, padding='same'))

G.add(BatchNormalization(momentum=0.9))

G.add(Activation('relu'))

G.add(Conv2DTranspose(int(depth/8), 5, padding='same'))

G.add(BatchNormalization(momentum=0.9))

G.add(Activation('relu'))

# Out: 28 x 28 x 1 grayscale image [0.0,1.0] per pix

G.add(Conv2DTranspose(1, 5, padding='same'))

G.add(Activation('sigmoid'))

G.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 12544)             1266944   
_________________________________________________________________
batch_normalization_1 (Batch (None, 12544)             50176     
_________________________________________________________________
activation_2 (Activation)    (None, 12544)             0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 256)         0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 7, 7, 256)         0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 14, 14, 256)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 14, 14, 128)       819328    
__________

Now we will deifne the optimizer for the Discriminator model, one of the tutorials above said that RMSprop worked better that Adam, so we used it

In [10]:
optimizer1 = RMSprop(lr=0.0008, clipvalue=1.0, decay=6e-8)
D.compile(loss='binary_crossentropy', optimizer=optimizer1,metrics=['accuracy'])

Now we will define the adverserial model in which both the generator and discriminator are stacked, we are using a neat trick from keras in which we can stck networks like we are stacking layers, and any update in the adversrial model will affect the generator and discriminator, and any update in the generator or discriminator will affect the adverserial model.

In [11]:
optimizer2 = RMSprop(lr=0.0004, clipvalue=1.0, decay=3e-8)
AM = Sequential()
AM.add(G)  #stacking the generator
AM.add(D)  #stacking the discriminator 
AM.compile(loss='binary_crossentropy', optimizer=optimizer2, metrics=['accuracy'])
D.trainable = False   #this line isn't important now and you can remove it, it just shows that the discriminator can be untrainable
AM.summary()
# print(AM.layers[1].name)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_2 (Sequential)    (None, 28, 28, 1)         2394241   
_________________________________________________________________
sequential_1 (Sequential)    (None, 1)                 4311553   
Total params: 6,705,794
Trainable params: 2,368,705
Non-trainable params: 4,337,089
_________________________________________________________________


## Training

In [12]:
#a function used to plot and save the output of the generator, it can take in a noise vector or generate one itself
def plot_images(save2file=False, fake=True, samples=16, noise=None, step=0):
    filename = 'mnist.png'
    if fake:
        if noise is None:
            noise = np.random.uniform(-1.0, 1.0, size=[samples, 100])
        else:
            filename = "mnist_%d.png" % step
        images = G.predict(noise)
    else:
        i = np.random.randint(0, self.x_train.shape[0], samples)
        images = x_train[i, :, :, :]

    plt.figure(figsize=(10,10))
    for i in range(images.shape[0]):
        plt.subplot(4, 4, i+1)
        image = images[i, :, :, :]
        image = np.reshape(image, [train_data_shape[1], train_data_shape[2]])
        plt.imshow(image, cmap='gray')
        plt.axis('off')
    plt.tight_layout()
    if save2file:
        plt.savefig(filename)
        plt.close('all')
    else:
        plt.show()

In [13]:
#some hyper parameters
save_interval = 500   #save a plot that includes the output of the generator every number of steps 
train_steps=2000      # end number of steps
start_step = 0        # start number of steps
batch_size=256        # batch size

pretraining the discriminator:

In [13]:
images_train = x_train[np.random.randint(0,x_train.shape[0], size=20000), :, :, :]  #selecting random 20k images from training data
noise = np.random.uniform(-1.0, 1.0, size=[20000, 100])   #generating random 20k noise vectors
images_fake = G.predict(noise)   #generating fake images from the generator
x = np.concatenate((images_train, images_fake))  #stacking both real and fake images
#generating labels in which real=1, fake =0
y = np.ones([2*20000, 1])
y[20000:, :] = 0

x,y = shuffle(x,y)  #shuffling the data and labels

D.trainable = True  #making sure that discriminator is trainable
d_loss = D.fit(x, y, batch_size=batch_size, epochs=1, validation_split=0.15) #the pretraining step

Train on 34000 samples, validate on 6000 samples
Epoch 1/1
34000/34000 [==============================] - 30s - loss: 0.1064 - acc: 0.9801 - val_loss: 3.8030e-06 - val_acc: 1.0000


The training procedure:

In [16]:
#noise input for generator output plotting
noise_input = None
if save_interval>0:
    noise_input = np.random.uniform(-1.0, 1.0, size=[16, 100])
#the training loop    
for i in range(start_step,train_steps):
    #training the discriminator
    images_train = x_train[np.random.randint(0,x_train.shape[0], size=batch_size), :, :, :] #selecting random real images whose count equals the batch size
    noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 100]) # generating the noise vectors whose count equals batch size
    images_fake = G.predict(noise) #generate fake images from generator
    x = np.concatenate((images_train, images_fake)) #stack real and fake images
    #generating the labels
    y = np.ones([2*batch_size, 1]) 
    y[batch_size:, :] = 0
    
    x,y = shuffle(x,y) #shuffling the images and labels
    D.trainable = True #make sure that the discriminator is trainanble
    d_loss = D.train_on_batch(x, y) #train the discriminator on this batch of data
    
    #training the adverserial model, aka training the generator
    y = np.ones([batch_size, 1]) #the labels, they are always one
    noise = np.random.uniform(-1.0, 1.0, size=[batch_size, 100]) #input noise for generator
    D.trainable = False #freeze the discirminator
    a_loss = AM.train_on_batch(noise, y) #train the adeveserial model, with the discrimimator frozen only the generator is updated\
                                         # and with all the labels equal (1), we are telling the generator to strive to generate data that will make the]
                                         # discriminator always output (1) aka fooling it.
    #print the losses and accuracy values 
    log_mesg = "%d: [D loss: %f, acc: %f]" % (i, d_loss[0], d_loss[1])
    log_mesg = "%s  [A loss: %f, acc: %f]" % (log_mesg, a_loss[0], a_loss[1])
    print(log_mesg)
    #save the plots of generator outputs
    if save_interval>0:
        if (i+1)%save_interval==0:
            plot_images(save2file=True, samples=noise_input.shape[0],noise=noise_input, step=(i+1))

535: [D loss: 0.692361, acc: 0.531250]  [A loss: 0.856972, acc: 0.156250]
536: [D loss: 0.684546, acc: 0.527344]  [A loss: 0.868491, acc: 0.140625]
537: [D loss: 0.691256, acc: 0.535156]  [A loss: 0.906451, acc: 0.117188]
538: [D loss: 0.679295, acc: 0.582031]  [A loss: 0.869319, acc: 0.187500]
539: [D loss: 0.696165, acc: 0.542969]  [A loss: 0.946788, acc: 0.093750]
540: [D loss: 0.764887, acc: 0.492188]  [A loss: 1.074614, acc: 0.031250]
541: [D loss: 0.701777, acc: 0.523438]  [A loss: 0.718632, acc: 0.460938]
542: [D loss: 0.792859, acc: 0.484375]  [A loss: 1.101982, acc: 0.007812]
543: [D loss: 0.699216, acc: 0.503906]  [A loss: 0.770990, acc: 0.273438]
544: [D loss: 0.731849, acc: 0.445312]  [A loss: 0.878569, acc: 0.062500]
545: [D loss: 0.701624, acc: 0.539062]  [A loss: 0.771065, acc: 0.265625]
546: [D loss: 0.705065, acc: 0.507812]  [A loss: 0.799459, acc: 0.218750]
547: [D loss: 0.709365, acc: 0.523438]  [A loss: 0.795836, acc: 0.203125]
548: [D loss: 0.688226, acc: 0.535156]

646: [D loss: 0.721429, acc: 0.480469]  [A loss: 0.848438, acc: 0.132812]
647: [D loss: 0.693031, acc: 0.539062]  [A loss: 0.820052, acc: 0.273438]
648: [D loss: 0.684155, acc: 0.554688]  [A loss: 0.854570, acc: 0.195312]
649: [D loss: 0.693051, acc: 0.554688]  [A loss: 0.844370, acc: 0.234375]
650: [D loss: 0.719260, acc: 0.511719]  [A loss: 0.827939, acc: 0.265625]
651: [D loss: 0.712193, acc: 0.519531]  [A loss: 0.917550, acc: 0.109375]
652: [D loss: 0.694300, acc: 0.515625]  [A loss: 0.808841, acc: 0.242188]
653: [D loss: 0.714602, acc: 0.492188]  [A loss: 0.883237, acc: 0.179688]
654: [D loss: 0.691032, acc: 0.566406]  [A loss: 0.768171, acc: 0.281250]
655: [D loss: 0.706196, acc: 0.507812]  [A loss: 0.979364, acc: 0.078125]
656: [D loss: 0.718573, acc: 0.496094]  [A loss: 0.888513, acc: 0.085938]
657: [D loss: 0.691185, acc: 0.558594]  [A loss: 0.832599, acc: 0.203125]
658: [D loss: 0.696828, acc: 0.523438]  [A loss: 0.843319, acc: 0.179688]
659: [D loss: 0.701121, acc: 0.492188]

757: [D loss: 0.696717, acc: 0.511719]  [A loss: 0.871542, acc: 0.218750]
758: [D loss: 0.694193, acc: 0.519531]  [A loss: 0.839393, acc: 0.195312]
759: [D loss: 0.712574, acc: 0.500000]  [A loss: 0.847031, acc: 0.171875]
760: [D loss: 0.706660, acc: 0.480469]  [A loss: 0.827273, acc: 0.203125]
761: [D loss: 0.707331, acc: 0.519531]  [A loss: 0.935710, acc: 0.125000]
762: [D loss: 0.712250, acc: 0.496094]  [A loss: 0.800128, acc: 0.367188]
763: [D loss: 0.707750, acc: 0.535156]  [A loss: 0.935853, acc: 0.101562]
764: [D loss: 0.683483, acc: 0.542969]  [A loss: 0.842669, acc: 0.218750]
765: [D loss: 0.703163, acc: 0.519531]  [A loss: 0.893234, acc: 0.140625]
766: [D loss: 0.680632, acc: 0.582031]  [A loss: 0.809721, acc: 0.273438]
767: [D loss: 0.688045, acc: 0.546875]  [A loss: 0.924874, acc: 0.132812]
768: [D loss: 0.705710, acc: 0.523438]  [A loss: 0.845727, acc: 0.171875]
769: [D loss: 0.680725, acc: 0.566406]  [A loss: 0.842746, acc: 0.234375]
770: [D loss: 0.705359, acc: 0.554688]

868: [D loss: 0.704372, acc: 0.527344]  [A loss: 0.920008, acc: 0.125000]
869: [D loss: 0.711200, acc: 0.542969]  [A loss: 0.909789, acc: 0.117188]
870: [D loss: 0.711557, acc: 0.511719]  [A loss: 1.094824, acc: 0.046875]
871: [D loss: 0.716648, acc: 0.500000]  [A loss: 0.760155, acc: 0.414062]
872: [D loss: 0.720651, acc: 0.539062]  [A loss: 0.986546, acc: 0.109375]
873: [D loss: 0.706643, acc: 0.531250]  [A loss: 0.863958, acc: 0.273438]
874: [D loss: 0.689646, acc: 0.535156]  [A loss: 0.955192, acc: 0.093750]
875: [D loss: 0.700102, acc: 0.503906]  [A loss: 0.901824, acc: 0.140625]
876: [D loss: 0.704695, acc: 0.515625]  [A loss: 0.923629, acc: 0.140625]
877: [D loss: 0.682131, acc: 0.597656]  [A loss: 0.877668, acc: 0.210938]
878: [D loss: 0.709509, acc: 0.507812]  [A loss: 0.902435, acc: 0.140625]
879: [D loss: 0.708672, acc: 0.488281]  [A loss: 0.933693, acc: 0.171875]
880: [D loss: 0.706985, acc: 0.574219]  [A loss: 0.860530, acc: 0.179688]
881: [D loss: 0.711894, acc: 0.523438]

979: [D loss: 0.787863, acc: 0.507812]  [A loss: 1.114728, acc: 0.023438]
980: [D loss: 0.708338, acc: 0.484375]  [A loss: 0.847922, acc: 0.171875]
981: [D loss: 0.746037, acc: 0.468750]  [A loss: 1.213970, acc: 0.007812]
982: [D loss: 0.700380, acc: 0.519531]  [A loss: 0.682686, acc: 0.492188]
983: [D loss: 0.790000, acc: 0.496094]  [A loss: 1.171542, acc: 0.015625]
984: [D loss: 0.711893, acc: 0.535156]  [A loss: 0.748947, acc: 0.398438]
985: [D loss: 0.738160, acc: 0.515625]  [A loss: 0.931946, acc: 0.125000]
986: [D loss: 0.710318, acc: 0.511719]  [A loss: 0.933005, acc: 0.078125]
987: [D loss: 0.694620, acc: 0.531250]  [A loss: 0.802888, acc: 0.289062]
988: [D loss: 0.719710, acc: 0.539062]  [A loss: 0.822587, acc: 0.242188]
989: [D loss: 0.708716, acc: 0.480469]  [A loss: 0.783071, acc: 0.320312]
990: [D loss: 0.703939, acc: 0.515625]  [A loss: 0.868922, acc: 0.148438]
991: [D loss: 0.707802, acc: 0.492188]  [A loss: 0.824824, acc: 0.328125]
992: [D loss: 0.715902, acc: 0.511719]

...

1529: [D loss: 0.699744, acc: 0.523438]  [A loss: 0.816606, acc: 0.257812]
1530: [D loss: 0.718599, acc: 0.515625]  [A loss: 0.943535, acc: 0.085938]
1531: [D loss: 0.685029, acc: 0.550781]  [A loss: 0.760915, acc: 0.429688]
1532: [D loss: 0.708587, acc: 0.562500]  [A loss: 0.912861, acc: 0.156250]
1533: [D loss: 0.688276, acc: 0.570312]  [A loss: 0.756723, acc: 0.414062]
1534: [D loss: 0.732977, acc: 0.519531]  [A loss: 0.968057, acc: 0.085938]
1535: [D loss: 0.708628, acc: 0.500000]  [A loss: 0.787682, acc: 0.335938]
1536: [D loss: 0.714440, acc: 0.535156]  [A loss: 0.921246, acc: 0.164062]
1537: [D loss: 0.675765, acc: 0.574219]  [A loss: 0.778306, acc: 0.312500]
1538: [D loss: 0.730222, acc: 0.511719]  [A loss: 0.884029, acc: 0.148438]
1539: [D loss: 0.697454, acc: 0.531250]  [A loss: 0.855252, acc: 0.148438]
1540: [D loss: 0.705949, acc: 0.531250]  [A loss: 0.864674, acc: 0.132812]
1541: [D loss: 0.706623, acc: 0.507812]  [A loss: 0.939075, acc: 0.054688]
1542: [D loss: 0.679883, 

1639: [D loss: 0.687302, acc: 0.523438]  [A loss: 0.884028, acc: 0.257812]
1640: [D loss: 0.744604, acc: 0.488281]  [A loss: 1.079618, acc: 0.054688]
1641: [D loss: 0.696811, acc: 0.523438]  [A loss: 0.922093, acc: 0.171875]
1642: [D loss: 0.697684, acc: 0.515625]  [A loss: 0.837895, acc: 0.226562]
1643: [D loss: 0.696625, acc: 0.539062]  [A loss: 0.961673, acc: 0.085938]
1644: [D loss: 0.722324, acc: 0.437500]  [A loss: 0.872648, acc: 0.218750]
1645: [D loss: 0.700663, acc: 0.542969]  [A loss: 0.935351, acc: 0.117188]
1646: [D loss: 0.691895, acc: 0.550781]  [A loss: 0.917342, acc: 0.109375]
1647: [D loss: 0.687965, acc: 0.558594]  [A loss: 0.899354, acc: 0.156250]
1648: [D loss: 0.662428, acc: 0.617188]  [A loss: 0.801090, acc: 0.320312]
1649: [D loss: 0.741364, acc: 0.492188]  [A loss: 1.092568, acc: 0.031250]
1650: [D loss: 0.715363, acc: 0.496094]  [A loss: 0.720811, acc: 0.468750]
1651: [D loss: 0.724034, acc: 0.480469]  [A loss: 0.994215, acc: 0.085938]
1652: [D loss: 0.683766, 

1749: [D loss: 0.691272, acc: 0.562500]  [A loss: 0.800275, acc: 0.312500]
1750: [D loss: 0.721243, acc: 0.496094]  [A loss: 1.086576, acc: 0.039062]
1751: [D loss: 0.678593, acc: 0.597656]  [A loss: 0.751681, acc: 0.406250]
1752: [D loss: 0.755202, acc: 0.539062]  [A loss: 1.067536, acc: 0.039062]
1753: [D loss: 0.690077, acc: 0.539062]  [A loss: 0.758669, acc: 0.359375]
1754: [D loss: 0.725100, acc: 0.507812]  [A loss: 1.036907, acc: 0.062500]
1755: [D loss: 0.714034, acc: 0.535156]  [A loss: 0.823295, acc: 0.289062]
1756: [D loss: 0.713559, acc: 0.496094]  [A loss: 0.937901, acc: 0.117188]
1757: [D loss: 0.696967, acc: 0.554688]  [A loss: 0.822366, acc: 0.242188]
1758: [D loss: 0.712851, acc: 0.531250]  [A loss: 0.924439, acc: 0.085938]
1759: [D loss: 0.704863, acc: 0.507812]  [A loss: 0.783138, acc: 0.343750]
1760: [D loss: 0.722710, acc: 0.554688]  [A loss: 0.956832, acc: 0.085938]
1761: [D loss: 0.700251, acc: 0.527344]  [A loss: 0.743890, acc: 0.421875]
1762: [D loss: 0.731578, 

1859: [D loss: 0.690575, acc: 0.523438]  [A loss: 0.906655, acc: 0.195312]
1860: [D loss: 0.745376, acc: 0.488281]  [A loss: 1.031288, acc: 0.093750]
1861: [D loss: 0.694008, acc: 0.550781]  [A loss: 0.663348, acc: 0.593750]
1862: [D loss: 0.770850, acc: 0.511719]  [A loss: 1.036101, acc: 0.062500]
1863: [D loss: 0.690556, acc: 0.582031]  [A loss: 0.793695, acc: 0.257812]
1864: [D loss: 0.801046, acc: 0.496094]  [A loss: 1.027561, acc: 0.054688]
1865: [D loss: 0.710016, acc: 0.503906]  [A loss: 0.857189, acc: 0.171875]
1866: [D loss: 0.713946, acc: 0.535156]  [A loss: 1.019423, acc: 0.078125]
1867: [D loss: 0.697552, acc: 0.503906]  [A loss: 0.673315, acc: 0.546875]
1868: [D loss: 0.783780, acc: 0.480469]  [A loss: 1.053186, acc: 0.031250]
1869: [D loss: 0.693267, acc: 0.531250]  [A loss: 0.725717, acc: 0.453125]
1870: [D loss: 0.745642, acc: 0.500000]  [A loss: 0.914639, acc: 0.148438]
1871: [D loss: 0.702961, acc: 0.546875]  [A loss: 0.804973, acc: 0.320312]
1872: [D loss: 0.726880, 

1969: [D loss: 0.727608, acc: 0.511719]  [A loss: 1.081923, acc: 0.039062]
1970: [D loss: 0.740184, acc: 0.496094]  [A loss: 0.898437, acc: 0.210938]
1971: [D loss: 0.710195, acc: 0.511719]  [A loss: 0.978609, acc: 0.148438]
1972: [D loss: 0.714699, acc: 0.500000]  [A loss: 0.847535, acc: 0.281250]
1973: [D loss: 0.719185, acc: 0.511719]  [A loss: 1.027329, acc: 0.109375]
1974: [D loss: 0.696515, acc: 0.546875]  [A loss: 0.800310, acc: 0.367188]
1975: [D loss: 0.715234, acc: 0.496094]  [A loss: 1.002603, acc: 0.125000]
1976: [D loss: 0.673701, acc: 0.566406]  [A loss: 0.814278, acc: 0.343750]
1977: [D loss: 0.718219, acc: 0.527344]  [A loss: 0.953179, acc: 0.148438]
1978: [D loss: 0.749476, acc: 0.488281]  [A loss: 1.076010, acc: 0.117188]
1979: [D loss: 0.721968, acc: 0.515625]  [A loss: 0.733479, acc: 0.429688]
1980: [D loss: 0.779682, acc: 0.488281]  [A loss: 1.217067, acc: 0.007812]
1981: [D loss: 0.695386, acc: 0.535156]  [A loss: 0.739433, acc: 0.429688]
1982: [D loss: 0.744543, 